<a href="https://colab.research.google.com/github/sktoyo/miscellaneous_codes/blob/master/GEO_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import GEOparse
import pandas as pd
import pylab as pl
import seaborn as sns
import numpy as np
import pandas as pd
import networkx as nx
import scipy
import json
import itertools

In [ ]:
gse = GEOparse.get_GEO(geo="GSE7621", destdir="gene_expression_data/")

In [33]:
def get_gse_file(geo_number, download, file_name):
  if download:
    gse = GEOparse.get_GEO(geo=geo_number, destdir = "/content/mnt/My Drive/Colab Notebooks/Projects/DCEN/data/gene_expression")
  else:
    gse = GEOparse.get_GEO(filepath = "/content/mnt/My Drive/Colab Notebooks/Projects/DCEN/data/gene_expression/%s" % file_name)
  return gse

In [91]:
def filter_genes(gse_obj, cut_off, log2, condition_list):
  if log2:
    pivoted_samples = gse_obj.pivot_samples('VALUE')[condition_list]
    return pivoted_samples
  else:
    pivoted_samples = np.log2(gse_obj.pivot_samples('VALUE')[condition_list])
  pivoted_samples_average = pivoted_samples.median(axis=1)
  print("Number of probes before filtering: ", len(pivoted_samples_average))
  expression_threshold = pivoted_samples_average.quantile(cut_off)
  expressed_probes = pivoted_samples_average[pivoted_samples_average >= expression_threshold]
  samples = np.log2(gse.pivot_samples("VALUE").loc[expressed_probes.keys()])
  samples = samples[condition_list]
  print("Number of probes after filtering: ", len(samples))
  return samples

In [119]:
def get_ttest(control_samples, post_samples):
  ttest_result = scipy.stats.ttest_ind(control_samples, post_samples, axis=1)
  ttest = pd.DataFrame({"stat": ttest_result[0], "pvalue": ttest_result[1]}, index=control_samples.index)

  return ttest

In [168]:
sample_ids = sorted([key for key in gse.gsms.keys()])

In [92]:
normal_samples = sample_ids[:9]
parkinson_smaples = sample_ids[9:]
normal_samples = filter_genes(gse, 0.25, True, normal_samples)
parkinson_smaples = filter_genes(gse, 0.25, True, parkinson_smaples)

In [121]:
ttest_df = get_ttest(normal_samples, parkinson_smaples)

D:\dev-env\envs\python36tf\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
D:\dev-env\envs\python36tf\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
D:\dev-env\envs\python36tf\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [124]:
ttest_df

,stat,pvalue
ID_REF,,
1007_s_at,0.582116,0.566152
1053_at,1.982402,0.059505
117_at,-2.178472,0.039874
121_at,1.817269,0.082229
1255_g_at,2.078169,0.049040
...,...,...
AFFX-r2-Ec-bioC-5_at,3.259104,0.003454
AFFX-r2-Ec-bioD-3_at,-0.531785,0.599974
AFFX-r2-Ec-bioD-5_at,-2.039331,0.053068


In [123]:
from statsmodels.stats import multitest

In [148]:
def get_FDR(ttest_df):
  corrected_pvalue = multitest.multipletests(pvals=ttest_df['pvalue'], method='bonferroni', alpha=0.05)
  print(corrected_pvalue)
  FDR = pd.DataFrame({'Rejected': corrected_pvalue[0], 'FDR': corrected_pvalue[1]}, index= ttest_df.index)

  return FDR

In [149]:
FDR = get_FDR(ttest_df)

(array([False, False, False, ..., False, False, False]), array([1., 1., 1., ..., 1., 1., 1.]), 9.43949468701355e-07, 9.201494322678003e-07)


D:\dev-env\envs\python36tf\lib\site-packages\statsmodels\stats\multitest.py:152: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
D:\dev-env\envs\python36tf\lib\site-packages\statsmodels\stats\multitest.py:256: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [155]:
selected = FDR.loc[FDR['FDR'] < 0.1]

In [159]:
def get_selected_df(gse_obj, selected_FDR):
  pivoted_samples = gse_obj.pivot_samples('VALUE').loc[selected_FDR.index]
  return pivoted_samples

In [160]:
selected_df = get_selected_df(gse, selected)

In [162]:
selected_df = np.log2(selected_df)

In [169]:
normal_samples = sample_ids[:9]
parkinson_smaples = sample_ids[9:]

In [170]:
normal_samples = selected_df[normal_samples]
parkinson_smaples = selected_df[parkinson_smaples]

In [171]:
LFCs = get_lfc(normal_samples, parkinson_smaples)

In [172]:
LFCs

,LFC
201436_at,-0.665640
223766_at,-0.890947
228845_at,-1.393827


In [93]:
def get_lfc(control_samples, post_samples):
  control_mean = control_samples.mean(axis=1)
  post_mean = post_samples.mean(axis=1)
  LFCs = pd.DataFrame({"LFC": (post_mean - control_mean).to_dict()})
  return LFCs

In [94]:
LFCs = get_lfc(normal_samples, parkinson_smaples)

In [112]:
def get_annotation(gene_dataframe, data_flatform, leftkey):
  gene_annotated = gene_dataframe.reset_index().merge(gse.gpls[data_flatform].table[["ID", "ENTREZ_GENE_ID", "Gene Symbol"]],
                                left_on=leftkey, right_on="ID").set_index(leftkey)
  del gene_annotated["ID"]
  # remove probes without ENTREZ
  gene_annotated = gene_annotated.dropna(subset=["ENTREZ_GENE_ID"])
  # remove probes with more than one gene assigned
  gene_annotated['ENTREZ_GENE_ID'] = pd.to_numeric(gene_annotated['ENTREZ_GENE_ID'], errors="coerce")
  gene_annotated.dropna(how="any", inplace=True)
  gene_annotated['ENTREZ_GENE_ID'] = gene_annotated.ENTREZ_GENE_ID.astype('int').astype('str')
  # for each gene average LFC over probes
  gene_annotated = gene_annotated.groupby("Gene Symbol").median()

  return gene_annotated

In [173]:
LFCs_annotated = get_annotation(LFCs, 'GPL570', 'index')

In [174]:
LFCs_annotated

,LFC
Gene Symbol,
EIF4E,-0.665640
LOC100133130,-0.890947
PLBD2,-1.393827
